In [1]:
import pandas as pd
import numpy as np
import time
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import string 
import helper_functions as hf
from selenium import webdriver

In [2]:
cwd=Path.cwd().resolve().parents[0]

In [3]:
HotAndNew = pd.read_csv(os.path.join(cwd, "Data", "HotAndNew.csv"))
print(HotAndNew.shape)
HotAndNew.head()

(47, 4)


,restaurant,category,stars,location
0,Tiger Sugar,Bubble Tea,4.0,348A Yonge Street
1,Chick-fil-A,Fast Food,4.5,709 Yonge Street
2,Phat Kaphrao,Thai,4.5,3016B Bathurst Street
3,Azkadenya - Mezza Diner,Middle Eastern,4.5,235 Queen Street West
4,EAT BKK,Thai,4.5,580 Queen Street W


In [4]:
BlogTO = pd.read_csv(os.path.join(cwd, "Data", "BlogTO_mentions.csv"))
print(BlogTO.shape)
BlogTO.head()

(424, 1)


,titles
0,Popular underground bar is closing after 12 ye...
1,10 great restaurants for Sunday brunch in Toronto
2,Toronto to become one of fourteen Good Food Ci...
3,"Toronto Restaurant Openings: YGO Lab, Chiang R..."
4,Soufi's announces they're reopening their Toro...


In [5]:
def remove_punctuation(title):
    for punctuation in string.punctuation:
        title= title.replace(punctuation," ")
    title=title.lower()
    return title

In [6]:
def count_in_list(title_list,word):
    temp_list=[token for token in title_list if remove_punctuation(word) in remove_punctuation(token)]
    if len(temp_list)!=0:
        print("'{0}' has been mentioned in:".format(word))
        print(temp_list)
        print(" ")
    return len([token for token in title_list if remove_punctuation(word) in remove_punctuation(token)])

In [7]:
HotAndNew["mention_count"]=HotAndNew["restaurant"].apply(lambda x:count_in_list(BlogTO["titles"],x))

'Tiger Sugar' has been mentioned in:
['Tiger Sugar Toronto']
 
'Chick-fil-A' has been mentioned in:
['There are still massive lineups at the new Chick-fil-A in Toronto', "This is what it was like inside Toronto's first Chick-fil-A on opening day", 'Protestors storm Chick-fil-A opening in Toronto', 'Christian group plans counter-protest in support of Toronto Chick-fil-A', 'People waited in line overnight for the opening of Chick-fil-A in Toronto', "The protest at Chick-fil-A's Toronto store opening is going to be massive", "Mouse spotted inside Chick-fil-A's first Toronto location", "There's a huge protest planned for the Chick-fil-A opening in Toronto", 'This is when the first Chick-fil-A location is opening in Toronto', 'People are already planning to boycott Chick-fil-A when it opens in Toronto', "Toronto's first Chick-fil-A location is about to open"]
 
'Phat Kaphrao' has been mentioned in:
['Phat Kaphrao']
 
'Bubble Lee' has been mentioned in:
['Bubble Lee Toronto']
 
'Vicino Itali

In [8]:
HotAndNew[HotAndNew["mention_count"]!=0].sort_values(by='mention_count', ascending=False).head()

,restaurant,category,stars,location,mention_count
1,Chick-fil-A,Fast Food,4.5,709 Yonge Street,11
46,Jollibee,Filipino,3.0,79 Billy Bishop Way,4
42,est Restaurant,Canadian (New),3.0,729 Queen Street E,2
11,Bloom,Cafes,4.0,596 Yonge street,2
35,Project Glyph,Desserts,5.0,40 Hayden Street,2


In [9]:
subway = pd.read_csv(os.path.join(cwd, "Data", "Torornto_Subway_Station_List.csv"))

In [10]:
my_addresses=["153 Aspen Hills Villas, Calgary", "1800 McGill College Avenue, Montreal", "40 king street west, Toronto"]

In [11]:
temp_df=subway[["station"]].copy()
temp_df["full_address"]=temp_df["station"].apply(lambda x: x + " Subway Station, Toronto, Canada")

In [12]:
jobs=8
mp_scraper=hf.mp_address_latlong(address_list=temp_df["full_address"].tolist(),
                                 patient_time=1,
                                 jobs=jobs)
start_time = time.time()
lat_long_list=mp_scraper.excecute()
end_time = time.time() - start_time
print("{0} address in {1} sec with {2} threads".format(len(temp_df["full_address"].tolist()), end_time, jobs))

74 address in 678.5671646595001 sec with 8 threads


In [13]:
subway["latitude"]=np.asarray(lat_long_list)[:,0].tolist()
subway["longitude"]=np.asarray(lat_long_list)[:,1].tolist()

In [14]:
subway

,station,line_number,Line_Color,latitude,longitude
0,Finch,1,Yellow,43.782083,-79.416066
1,North York Centre,1,Yellow,43.768595,-79.412732
2,Sheppard-Yonge,1,Yellow,43.761505,-79.410989
3,York Mills,1,Yellow,43.744069,-79.406720
4,Lawrence,1,Yellow,43.726597,-79.402592
...,...,...,...,...,...
69,Sheppard-Yonge,4,Purple,43.761505,-79.410989
70,Bayview,4,Purple,43.767022,-79.386758
71,Bessarion,4,Purple,43.769341,-79.375934
72,Leslie,4,Purple,43.771098,-79.365997


In [15]:
temp_df=HotAndNew[["location"]].copy()
temp_df["full_address"]=temp_df["location"].apply(lambda x: x + ", Toronto, Canada")

In [16]:
jobs=8
mp_scraper=hf.mp_address_latlong(address_list=temp_df["full_address"].tolist(),
                                 patient_time=1,
                                 jobs=jobs)
start_time = time.time()
lat_long_list=mp_scraper.excecute()
end_time = time.time() - start_time
print("{0} address in {1} sec with {2} threads".format(len(temp_df["full_address"].tolist()), end_time, jobs))

47 address in 473.95118951797485 sec with 8 threads


In [17]:
HotAndNew["latitude"]=np.asarray(lat_long_list)[:,0].tolist()
HotAndNew["longitude"]=np.asarray(lat_long_list)[:,1].tolist()

In [19]:
HotAndNew.head()

,restaurant,category,stars,location,mention_count,latitude,longitude
0,Tiger Sugar,Bubble Tea,4.0,348A Yonge Street,1,43.658036,-79.381941
1,Chick-fil-A,Fast Food,4.5,709 Yonge Street,11,43.669547,-79.386144
2,Phat Kaphrao,Thai,4.5,3016B Bathurst Street,1,43.717842,-79.429744
3,Azkadenya - Mezza Diner,Middle Eastern,4.5,235 Queen Street West,0,43.650249,-79.388288
4,EAT BKK,Thai,4.5,580 Queen Street W,0,,


In [20]:
subway.to_csv(os.path.join(cwd, "Data", "Torornto_Subway_Station_List.csv"),index=False)

In [21]:
HotAndNew.to_csv(os.path.join(cwd, "Data", "BlogTO_mentions.csv"),index=False)